Impoting Libraries

In [1]:
import pandas as pd
import ollama
import numpy as np
import faiss

Loading Dataset

In [2]:
df = pd.read_csv("./Dataset/Training Dataset.csv")
df.fillna("", inplace=True)
df.head()

C:\Users\Devansh Kumar\AppData\Local\Temp\ipykernel_50300\2023626122.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna("", inplace=True)


,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [3]:
docs = []
for idx, row in df.iterrows():
    context = "\n".join([f"{col}: {row[col]}" for col in df.columns])
    docs.append({"id": idx, "text": context})

Embed Documents using "mxbai-embed-large" 

In [4]:
def get_embedding(text):
    response = ollama.embeddings(model='mxbai-embed-large', prompt=text)
    return np.array(response['embedding'])

# Embed all docs
doc_embeddings = [(doc['id'], get_embedding(doc['text'])) for doc in docs]

Store embeddings in a simple FAISS index

In [5]:
dim = len(doc_embeddings[0][1])
index = faiss.IndexFlatL2(dim)

ids = [doc_id for doc_id, emb in doc_embeddings]
vectors = np.array([emb for _, emb in doc_embeddings]).astype("float32")

index.add(vectors)

Query Pipeline (RAG)

In [6]:
def retrieve_top_k(query, k=3):
    query_emb = get_embedding(query).astype("float32").reshape(1, -1)
    D, I = index.search(query_emb, k)
    results = [docs[i] for i in I[0]]
    return results

Generate Response using "gemma:2b"

In [7]:
def generate_answer(query, context_docs):
    context = "\n\n".join([doc["text"] for doc in context_docs])
    prompt = f"""You are a helpful assistant answering questions about a loan dataset.

Context:
{context}

Question: {query}
Answer:"""

    response = ollama.chat(model="gemma:2b", messages=[{"role": "user", "content": prompt}])
    return response['message']['content']


Chatbot

In [8]:
while True:
    query = input("Ask a question: ")
    context_docs = retrieve_top_k(query)
    answer = generate_answer(query, context_docs)
    print(f"\n🔍 Answer: {answer}\n")


🔍 Answer: Sure, here's the answer to your question:

Loan_ID: LP001015
Gender: Male
Married: Yes
Dependents: 0
Education: Graduate
Self_Employed: No
ApplicantIncome: 5720
CoapplicantIncome: 0.0
LoanAmount: 74.0
Loan_Amount_Term: 360.0
Credit_History: 1.0
Property_Area: Urban
Loan_Status: N



AssertionError: 